In [2]:
import pandas as pd
import json
from main import load_base_model, load_yml
from models.dino import utils
from models.dino import vision_transformer as vits
from models.classification import linear
from pathlib import Path
from _types import Config
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# unpack log
log_data = []
log_path = "linear-dino-0/log.txt"
with open(log_path) as file:
    while line := file.readline():
        line = json.loads(line.rstrip())
        print(line)
        log_data.append(line)
df = pd.DataFrame(log_data)

FileNotFoundError: [Errno 2] No such file or directory: 'linear-dino-0/log.txt'

In [ ]:
df

In [5]:
ROOT = Path("/home/sepal-user/")
config_file = ROOT /"sig-ssl4eo/linear-dino-0.yml"
config = Config(**load_yml(config_file))
print(config) 

Config(imgs_training='/home/sepal-user/data_ssl4eo/data_ssl4eo/training/imgs', labels_training='/home/sepal-user/sig-ssl4eo/fao_models/data/match_training_sample.csv', imgs_testing='/home/sepal-user/data_ssl4eo/data_ssl4eo/testing/imgs', labels_testing='/home/sepal-user/sig-ssl4eo/fao_models/data/match_testing_sample.csv', arch='vit_small', model_root='/home/sepal-user/sig-ssl4eo/B13_vits16_dino_0099_ckpt.pth', avgpool_patchtokens=False, patch_size=16, n_last_blocks=4, lr=0.001, batch_size=32, checkpoints_dir='linear-dino-0', resume=True, epochs=5, num_workers=10, seed=42, random_subset_frac=0.01, checkpoint_key='teacher')


In [16]:
from datasets.ssl4eo_dataset import SSL4EO, random_subset
from torch.utils.data import DataLoader
_data_train = SSL4EO(
    root=config.imgs_training, 
    mode="s2c", 
    label=config.labels_training, 
    normalize=False
)
_data_test = SSL4EO(
    root=config.imgs_testing, 
    mode="s2c", 
    label=config.labels_testing, 
    normalize=False
)
_data_train = random_subset(_data_train, config.random_subset_frac, config.seed)
_data_test = random_subset(_data_train, config.random_subset_frac, config.seed)

val_loader = DataLoader(
    _data_test, batch_size=2, shuffle=False, drop_last=True
)

In [38]:
import torch
image, target = _data_test[0]
image = torch.unsqueeze(torch.tensor(image),0).type(torch.float32)
target = torch.unsqueeze(torch.tensor(target),0).type(torch.float32)

In [9]:

model, embed_dim = load_base_model(
    pretrained=config.model_root,
    checkpoint_key=config.checkpoint_key,
    arch=config.arch,
    patch_size=config.patch_size,
    n_last_blocks=config.n_last_blocks,
    avgpool_patchtokens=config.avgpool_patchtokens,
)
linear_classifier = linear.LinearClassifier(embed_dim, num_labels=2)

Take key teacher in provided checkpoint dict
Pretrained weights found at /home/sepal-user/sig-ssl4eo/B13_vits16_dino_0099_ckpt.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])
Model vit_small built.


In [31]:
linear_classifier.eval()
with torch.no_grad():
    intermediate_output = model.get_intermediate_layers(image, config.n_last_blocks)
    output = torch.cat([x[:, 0] for x in intermediate_output], dim=-1)

output = linear_classifier(output)

In [41]:
from sklearn.metrics import average_precision_score
score = torch.sigmoid(output).detach().cpu()
acc1 = average_precision_score(target, score, average="micro") * 100.0
score, target, acc1

(tensor([[0.4467, 0.3195]]), tensor([[0., 1.]]), 50.0)